<a href="https://colab.research.google.com/github/aynuod/JPA-Jenkins/blob/main/TraductionAutoEx1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1. Constituer le contexte du document en récupérant tous les termes significatifs**

In [12]:
import spacy

# Charger le modèle anglais de spaCy
nlp = spacy.load('en_core_web_sm')

# Texte à traduire
text = "Get busy living or get busy dying.You only live once, but if you do it right, once is enough."

# Traiter le texte
doc = nlp(text)

# Récupérer les termes significatifs (noms, adjectifs, verbes)
significant_terms = [token.text for token in doc if token.pos_ in ['NOUN', 'ADJ', 'VERB']]
print(significant_terms)


['Get', 'busy', 'living', 'get', 'busy', 'dying', 'live', 'do', 'enough']


**2. Découper le texte en des phrases simples et récupérer les tags de leurs mots**


In [13]:
# Découper le texte en phrases
sentences = list(doc.sents)

# Récupérer les tags de chaque mot dans chaque phrase
for sentence in sentences:
    for token in sentence:
        print(f'Token: {token.text}, POS: {token.pos_}, Tag: {token.tag_}')


Token: Get, POS: VERB, Tag: VB
Token: busy, POS: ADJ, Tag: JJ
Token: living, POS: VERB, Tag: VBG
Token: or, POS: CCONJ, Tag: CC
Token: get, POS: VERB, Tag: VB
Token: busy, POS: ADJ, Tag: JJ
Token: dying, POS: VERB, Tag: VBG
Token: ., POS: PUNCT, Tag: .
Token: You, POS: PRON, Tag: PRP
Token: only, POS: ADV, Tag: RB
Token: live, POS: VERB, Tag: VBP
Token: once, POS: ADV, Tag: RB
Token: ,, POS: PUNCT, Tag: ,
Token: but, POS: CCONJ, Tag: CC
Token: if, POS: SCONJ, Tag: IN
Token: you, POS: PRON, Tag: PRP
Token: do, POS: VERB, Tag: VBP
Token: it, POS: PRON, Tag: PRP
Token: right, POS: ADV, Tag: RB
Token: ,, POS: PUNCT, Tag: ,
Token: once, POS: ADV, Tag: RB
Token: is, POS: AUX, Tag: VBZ
Token: enough, POS: ADJ, Tag: JJ
Token: ., POS: PUNCT, Tag: .


**3. Pour chaque phrase, récupérer le sens exact de chaque terme en se basant sur leurs tags et leur contexte**

In [14]:
# Exemple simple de désambiguïsation de sens avec spaCy
for sentence in sentences:
    for token in sentence:
        print(f'Token: {token.text}, Lemma: {token.lemma_}, POS: {token.pos_}, Tag: {token.tag_}')


Token: Get, Lemma: get, POS: VERB, Tag: VB
Token: busy, Lemma: busy, POS: ADJ, Tag: JJ
Token: living, Lemma: live, POS: VERB, Tag: VBG
Token: or, Lemma: or, POS: CCONJ, Tag: CC
Token: get, Lemma: get, POS: VERB, Tag: VB
Token: busy, Lemma: busy, POS: ADJ, Tag: JJ
Token: dying, Lemma: die, POS: VERB, Tag: VBG
Token: ., Lemma: ., POS: PUNCT, Tag: .
Token: You, Lemma: you, POS: PRON, Tag: PRP
Token: only, Lemma: only, POS: ADV, Tag: RB
Token: live, Lemma: live, POS: VERB, Tag: VBP
Token: once, Lemma: once, POS: ADV, Tag: RB
Token: ,, Lemma: ,, POS: PUNCT, Tag: ,
Token: but, Lemma: but, POS: CCONJ, Tag: CC
Token: if, Lemma: if, POS: SCONJ, Tag: IN
Token: you, Lemma: you, POS: PRON, Tag: PRP
Token: do, Lemma: do, POS: VERB, Tag: VBP
Token: it, Lemma: it, POS: PRON, Tag: PRP
Token: right, Lemma: right, POS: ADV, Tag: RB
Token: ,, Lemma: ,, POS: PUNCT, Tag: ,
Token: once, Lemma: once, POS: ADV, Tag: RB
Token: is, Lemma: be, POS: AUX, Tag: VBZ
Token: enough, Lemma: enough, POS: ADJ, Tag: JJ
To

**4. Récupérer les termes correspondants en français**

In [15]:
from transformers import MarianMTModel, MarianTokenizer

# Charger le modèle de traduction anglais-français
model_name = 'Helsinki-NLP/opus-mt-en-fr'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# Traduire chaque phrase
for sentence in sentences:
    translated = model.generate(**tokenizer(sentence.text, return_tensors="pt", padding=True))
    translation = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]
    print(f'Original: {sentence.text}')
    print(f'Translation: {translation[0]}')


Original: Get busy living or get busy dying.
Translation: Occupez-vous de vivre ou de mourir.
Original: You only live once, but if you do it right, once is enough.
Translation: Tu ne vis qu'une fois, mais si tu le fais bien, une fois suffit.


**5. Pour chaque phrase, afficher à l’utilisateur les propositions de traduction pour les noms, les adjectifs et les verbes**

In [16]:
# Afficher les propositions de traduction pour les noms, adjectifs et verbes
for sentence in sentences:
    translated = model.generate(**tokenizer(sentence.text, return_tensors="pt", padding=True))
    translation = tokenizer.decode(translated[0], skip_special_tokens=True)
    print(f'Original: {sentence.text}')
    print(f'Translation: {translation}')

    # Traiter la traduction pour récupérer les termes significatifs
    translated_doc = nlp(translation)
    significant_terms_fr = [token.text for token in translated_doc if token.pos_ in ['NOUN', 'ADJ', 'VERB']]
    print(f'Significant terms in French: {significant_terms_fr}')


Original: Get busy living or get busy dying.
Translation: Occupez-vous de vivre ou de mourir.
Significant terms in French: ['Occupez', 'vous', 'vivre', 'mourir']
Original: You only live once, but if you do it right, once is enough.
Translation: Tu ne vis qu'une fois, mais si tu le fais bien, une fois suffit.
Significant terms in French: ['fois', 'si', 'une', 'fois', 'suffit']
